In [1]:
# import required library

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import concurrent.futures
import requests
import cloudscraper
import time
from requests.exceptions import RequestException

/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### earthquake dataset form usgs

In [2]:
#  Data collection begains with the downloading the dataset form USGS, the USGS provides science for a changing world, which reflects and responds to society’s continuously evolving needs. As the science arm of the Department of the Interior, the USGS brings an array of earth, water, biological, and mapping data and expertise to bear in support of decision-making on environmental, resource, and public safety issues. 
# url = 'https://earthquake.usgs.gov/earthquakes/search/#%7B%22currentfeatureid%22%3Anull%2C%22mapposition%22%3A%5B%5B-86.46756%2C-537.1875%5D%2C%5B86.44583%2C160.3125%5D%5D%2C%22autoUpdate%22%3A%5B%22autoUpdate%22%5D%2C%22feed%22%3A%22undefined_undefined%22%2C%22listFormat%22%3A%22default%22%2C%22restrictListToMap%22%3A%5B%5D%2C%22sort%22%3A%22newest%22%2C%22basemap%22%3A%22grayscale%22%2C%22overlays%22%3A%5B%22plates%22%5D%2C%22distanceUnit%22%3A%22km%22%2C%22timezone%22%3A%22local%22%2C%22viewModes%22%3A%5B%22settings%22%2C%22map%22%5D%2C%22event%22%3Anull%2C%22search%22%3Anull%7D'
# from this source ive downloaded dataset from year 2011 to present date which is 2024 jan.
# from the website we can download less than 20000 entries at a time and there are more thatn 20k lables in a year 
# so there are several dataset that i ve to import and concat.

In [3]:
# Function to read CSV files using pandas. where path is the location of the file in the local storage 

def read_csv(path):
    return pd.read_csv(path)



In [7]:
# To read all CSV files from a specific location using the glob.glob method 

csv_files = glob.glob('../dataset/first_dataset_from_sorce/*.{}'.format('csv'))
csv_files
# these are the list of usefull CSV dataframe 

['../dataset/first_dataset_from_sorce/query (11).csv',
 '../dataset/first_dataset_from_sorce/query (2).csv',
 '../dataset/first_dataset_from_sorce/query (27).csv',
 '../dataset/first_dataset_from_sorce/query (26).csv',
 '../dataset/first_dataset_from_sorce/query (3).csv',
 '../dataset/first_dataset_from_sorce/query (10).csv',
 '../dataset/first_dataset_from_sorce/query (21).csv',
 '../dataset/first_dataset_from_sorce/query (8).csv',
 '../dataset/first_dataset_from_sorce/query (17).csv',
 '../dataset/first_dataset_from_sorce/query (4).csv',
 '../dataset/first_dataset_from_sorce/query (5).csv',
 '../dataset/first_dataset_from_sorce/query (16).csv',
 '../dataset/first_dataset_from_sorce/query (9).csv',
 '../dataset/first_dataset_from_sorce/query (20).csv',
 '../dataset/first_dataset_from_sorce/query (19).csv',
 '../dataset/first_dataset_from_sorce/query (23).csv',
 '../dataset/first_dataset_from_sorce/query (6).csv',
 '../dataset/first_dataset_from_sorce/query (15).csv',
 '../dataset/firs

In [6]:
#concating all the csv dataset in to one 

df_concat = pd.concat([read_csv(f) for f in csv_files ], ignore_index=True)

In [7]:
# taking copy of the orginal dataframe 

df = df_concat.copy()
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [8]:
# sorting the dataframe by the time column because during concatining the files entries are ultered

df = df.sort_values(by='time').reset_index(drop = True)

In [9]:
df['time'] = df['time'].str.slice(0,19)

In [10]:
df['time'] = df['time'].str.replace('T',' ')

In [11]:
df.shape

(363898, 22)

In [12]:
# converting the time features in to the readable format to scrape distance of moon by the time 
df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')

In [13]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

### filtering the dataset based on requirement for the research 

In [14]:
# there are different types of recorded sesmic waves here we are working on earthquake dataset so removing oter types of waves
df = df[df['type'].isin(['earthquake'])]
df.shape


(360471, 22)

In [15]:
df = df.sort_values(by='time')

In [16]:
df = df[df['mag'] > 5]

### Webscraping Distance 

In [17]:
from skyfield.api import Loader, Topos
from datetime import datetime

def get_distance_earth_moon(date_time):
    # Load the ephemeris data
    load = Loader('~/skyfield-data')
    planets = load('de421.bsp')
    earth, moon = planets['earth'], planets['moon']

    # Get the position of the Moon relative to Earth
    ts = load.timescale()
    t = ts.utc(date_time.year, date_time.month, date_time.day, date_time.hour, date_time.minute, date_time.second)
    astrometric = (moon - earth).at(t)

    # Get the distance between Earth and Moon
    distance_km = astrometric.distance().km
#     print(distance_km)
    return distance_km



# date_time = datetime(2024, 5, 27, 12, 0, 0)  # Date and time you want to calculate the distance for

df['distance'] = df['time'].apply(get_distance_earth_moon)

In [18]:
df['distance'] = df['distance'].astype(int)

In [35]:
df.head(2)


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,horizontalError,depthError,magError,magNst,status,locationSource,magSource,distance,url,force
20,2011-01-01 09:56:58,-26.803,-63.136,576.8,7.00,mww,607.0,18.4,NaN,0.85,...,NaN,NaN,NaN,NaN,reviewed,us,us,380543,https://geodesy.noaa.gov/api/gravd/gp?lat=-26....,2.021809e+20
21,2011-01-01 09:59:36,-26.851,-63.217,592.9,5.78,mw,NaN,NaN,NaN,NaN,...,NaN,25.0,0.4,NaN,reviewed,iscgem,iscgem,380548,https://geodesy.noaa.gov/api/gravd/gp?lat=-26....,2.021755e+20


### webscraping gravity 

In [20]:
def get_url(lat, lon):
    url = str(f'https://geodesy.noaa.gov/api/gravd/gp?lat={lat}&lon={lon}&eht=100.0')
#     print(url)
    return url
get_url(df['latitude'].iloc[0], df['longitude'].iloc[0])

'https://geodesy.noaa.gov/api/gravd/gp?lat=-26.803&lon=-63.136&eht=100.0'

In [21]:
df['url'] = np.vectorize(get_url)(df['latitude'], df['longitude'])

In [24]:


# to fetch gravity data from multiple URLs concurrently using multithreading. The provided code defines a function get_gravity to retrieve gravity data from 
# a single URL and another function fetch_gravity_concurrently to apply multithreading for concurrent requests.

def get_gravity(url):
    scraper = cloudscraper.create_scraper()
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = scraper.get(url)
            if response.status_code == 200:
                data = response.json()  # Parse the JSON response
                return data.get('predictedGravity', 'none')
            else:
                print(f"Request failed with status code: {response.status_code}")
                time.sleep(180)  # Wait a bit before retrying
                return 'none'
        except RequestException as e:
            print(f"Attempt {attempt + 1} for URL {url} failed: {e}")
            if attempt == max_retries - 1:
                return 'none'
            
# Function to apply multithreading for concurrent requests
def fetch_gravity_concurrently(urls):
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_url = {executor.submit(get_gravity, url): url for url in urls}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                results.append(result)
                print(len(results), result)
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')
                results.append('none')
    return results

# Main execution
if __name__ == "__main__":
    urls = df['url'].tolist()
    df['gravity'] = fetch_gravity_concurrently(urls)

In [36]:

df.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,horizontalError,depthError,magError,magNst,status,locationSource,magSource,distance,url,force
20,2011-01-01 09:56:58,-26.803,-63.136,576.8,7.00,mww,607.0,18.4,NaN,0.85,...,NaN,NaN,NaN,NaN,reviewed,us,us,380543,https://geodesy.noaa.gov/api/gravd/gp?lat=-26....,2.021809e+20
21,2011-01-01 09:59:36,-26.851,-63.217,592.9,5.78,mw,NaN,NaN,NaN,NaN,...,NaN,25.0,0.4,NaN,reviewed,iscgem,iscgem,380548,https://geodesy.noaa.gov/api/gravd/gp?lat=-26....,2.021755e+20


### calculating force between earth and moon

In [27]:

# G is the universal gravitational constant 6.67 X 10-11N.m2/kg2.  which can be represented as 6.67e-11.
G = 6.67e-11
# the mass of moon in the kg
moon_mass =  7.35e22

# this is mass of the eart in the kg
earth_mass = 5.9722e24



In [28]:
# by usning sample dataframe the force acting at the time of the event of earth quake is calculated, 
# Calculate force
force = (G * earth_mass * moon_mass) / (df['distance'] * 1000) ** 2

# Assign the calculated force to the 'force' column using .loc
df.loc[:, 'force'] = force

In [29]:
# df.to_csv('final_dataset.csv')